In [6]:
case class Employee(name:String,department:String)

def lookupByName(name:String) : Employee = {
    name match {
        case "joe" => new Employee("joe","development")
        case _ => null
    }
}

def getDepartment(name:String) : String = {
    val employee = lookupByName(name) 
    if ( employee != null ) employee.department
    else ""
}

val dept: String = getDepartment("joe")
val dept2: String = getDepartment("jaehoon")

: 

+ 위 코드는 어떤 문제가 있나요? 문제를 찾아 봅시다.
+ 위 문제를 해결하는 방법은?
+ 위 코드를 함수형으로 바꾸려면 어떻게 해야 할까?
+ 위 코드를 함수형으로 바꾼 형태는 어떻게 될까?

In [7]:
case class Employee(name:String,department:String)

def lookupByName(name:String) : Employee = {
    name match {
        case "joe" => new Employee("joe","development")
    }
}

def getDepartment(name:String) : String = {
    lookupByName(name).department
}

val dept: String = getDepartment("joe")

defined class Employee
defined function lookupByName
defined function getDepartment
dept: String = "development"

# 4. 예외를 이용하지 않은 오류 처리

+ 이번 장의 목표
    + 실패 상황과 예외를 보통의 값으로 표현할 수 있다.
    + 일반적인 오류 처리-복구 패턴을 추상화한 고차 함수를 작성할 수 있다. 
+ 예외
    + 예외를 던지는 것은 부수효과중 하나다. 
    + 함수적 코드에서는 예외를 사용하지 않는다.
    + 함수적 코드에서는 예외를 어떻게 처리할까?
        + 함수적 코드에서는 예외를 값으로 돌려준다.
            + 이 방법은 예외를 던지는 방법보다 더 안전하다.
            + 이 방법은 참조 투명성을 깨뜨리지 않는다.
        + 고차 함수를 이용해서 "오류 처리 논리를 통합(consolidation)"할 수 있다.

## 4.1 예외의 장단점
+ 예외는 참조 투명성을 해칠까?
+ 예외가 참조 투명성을 해치는 것이 왜 문제가 될까?

In [8]:
def failingFn1(i:Int) : Int = {
    val y : Int = throw new Exception("fail")
    try {
        val x = 42 + 5
        x //x + y
    }
    catch { case e : Exception => 43 }
}

val x = failingFn1(1)

: 

In [9]:
def failingFn2(i:Int) : Int = {
    try {
        val x = 42 + 5
        x + ((throw new Exception("fail")): Int)
    }
    catch { case e : Exception => 43 }
}

val x = failingFn2(1)

defined function failingFn2
x: Int = 43

+ 참조 투명성(Referential Transparency)
    + 표현식이 참조 투명하다면, 문맥(context)에 의존하지 않으며 지역적으로 추론할 수 있다.
    + 표현식이 참조 투명하지 않다면, 표현식의 의미가 문맥에 의존적이고, 전역의 추론이 필요하다.
    + 표현식 42 + 5 는 문맥에 관계없이 항상 47이다.
    + 그러나, throw new Exception("fail") 의 의미는 문맥에 따라 달라진다.
        + try 블럭에 포함되어 있지 않으면, 예외를 던진다.
        + try 블럭에 포함되어 있다면, 오류 처리 로직(catch 문)이 실행된다. 
        
+ 예외의 문제
    + 예외는 참조 투명성을 위반하고, 문맥 의존성을 만든다. 
        + 치환 모형의 간단한 추론이 불가능해진다.
        + 예외에 기초한 혼란스런 코드가 만들어진다.
        + 예외는 오류 처리에만 사용하고, 흐름의 제어에는 사용하지 말아야 한다.
    + 예외는 형식에 안전하지 않다.
        + 함수의 형식(signature)만 보고는 이 함수가 예외를 던질 수 있다는 사실을 전혀 알 수 없다. 
        + 따라서, 컴파일러는 failingFn1 의 호출자가 예외를 처리하도록 강제할 방법이 없다.
        + failingFn1 의 호출자에서 예외 처리를 하지 않으면, 실행시점에야 발견된다.
        
        
+ Java 의 Checked Exception
    + 오류를 처리할 것인지, Call Stack 상의 아래 함수(예외가 발생한 함수를 부른 함수)로 전달할 것인지 강제화한다.
    + 호출하는 쪽에 판에 박힌 코드(boilerplate)가 만들어진다.
    + 고차함수에서는 통하지 않는다.
        + 고차함수는 함수를 인자로 받는다.
        + 이 함수가 어떤 예외를 발생시킬지 알 수가 없다.


+ 오류 처리 논리를 consolidation 하고 중앙집중화를 유지하려면 어떻게 해야할까?
    + 예외를 던지는 대신, 예외적인 조건이 발생했음을 뜻하는 값을 돌려준다.
        + C에서 예외 처리를 위해 오류부호를 돌려주는 방식.
    + 오류 부호를 직접 돌려주는 대신, 오류 부호를 돌려주기 위한 새로운 타입을 도입하고
    + 오류 처리와 전파에 관한 공통적인 패턴들을 고차 함수를 이용해 캡슐화한다. 
        + 형식에 완전히 안전하다.
        + 구문 잡음을 최소화할 수 있다.
        + 스칼라의 타입 검사기가 실수를 미리 발견할 수 있다.


        
## 4.2 예외의 가능한 대안들


In [66]:
def mean(xs: Seq[Double]) : Double = 
    if ( xs.isEmpty )
        throw new ArithmeticException("mean of empty list!")
    else xs.sum / xs.length

defined function mean

+ 평균값을 구하는 mean 함수를 보자.
    + 이 함수는 부분함수(partial function)이다. 
    + 일부 입력에 대해서는 정의되지 않는 함수를 부분함수라고 한다.
+ 예외를 처리하는 방법
    + 첫번째 대안. 
        + Double 형식의 가짜 값을 돌려준다. 
            + 경계값을 돌려준다. NaN 값을 돌려준다.
            + type 값대신 null 을 돌려준다. 
        + 이 접근 방식의 문제
            + 오류가 소리 없이 전파될 수 있다.
                + 오류 점검 코드를 빼먹어도 컴파일러가 경고해 줄 수 없다.
                + 오류가 발생하면 이후 코드가 제대로 작동되지 않는다.
                + 오류가 훨씬 나중에 다른 코드 영역에서 검출된다.
            + 호출하는 쪽에 오류를 검사하는 판에 박힌 코드들이 추가되어야 한다. 
            + 다형적 코드에는 적용할 수가 없다. 
                + 형식의 경계값을 결정하는 것이 불가능할 수도 있다.
            + 호출자에게 특별한 방침이나 호출 규약을 요구한다.
                + 호출자는 mean 을 호출해서 사용하는 것 이상의 작업을 해야 한다. 
                + 함수에 이런 특별한 방침을 요구한다면, 모든 인슈를 동일한 방식으로 처리해야 하는 고차 함수에 전달하기 어렵다.

+ 두번째 대안
    + 함수가 입력을 처리할 수 없을 때, 처리해야 하는 것을 인수로 전달한다. 
    + mean은 부분함수가 아니라 완전함수가 된다.
    + mean 함수의 호출자가 결과가 정의되지 않은 경우의 처리 방식을 호출하는 사람이 알고 있어야 한다.
    + 단점 
        + mean 이 정의되지 않은 입력에 대해서, 계산을 취소해야 한다면 어떻게 할까?
        + 더 큰 계산에서 지금과는 완전히 다른 분기로 넘어가야 한다면?
        + onEmpty 값만을 넘겨줘서는 그런 유연성을 얻을 수 없다.

In [67]:
def mean_1(xs: Seq[Double],onEmpty:Double) : Double = 
    if ( xs.isEmpty ) onEmpty
    else xs.sum / xs.length

defined function mean_1

## 4.3 Option 자료 형식

+ 오류처리 논리를 통합하는 해법
    + 반환 형식을 이용해서 함수가 항상 답을 내지 못한다는 점을 명시적으로 표현한다. 
    + 이 방법은 오류 처리 전략을 호출자에게 미루는 것으로 볼 수도 있다. 
    + 이 방법을 구현하기 위해서 Option 이라는 새로운 타입을 도입한다.

In [68]:
sealed trait Option[+A]
case class Some[+A](get:A) extends Option[A]
case object None extends Option[Nothing]

defined trait Option
defined class Some
defined object None

+ Option 의 값 해석하기
    + Some : 계산 결과를 정의할 수 있는 경우에 값을 감싸서 돌려준다.
    + None : 계산 결과를 정의할 수 없는 경우에 돌려준다.

In [69]:
def mean_1(xs: Seq[Double],onEmpty:Double) : Option[Double] = 
    if ( xs.isEmpty ) None
    else Some(xs.sum / xs.length)

defined function mean_1

+ 평균을 구하는 함수 mean 을 위와 같이 바꿀 수 있다.
    + 결과값이 Option 이기 때문에, 함수 결과가 정의되지 않는 경우가 있다는 것을 알 수 있다.
    + mean_1 은 항상 값을 돌려주기 때문에 완전함수가 된다.

### 4.3.1 Option 사용하기

+ Functional Programming 에서는 Option 과 같은 타입을 이용해서 부분함수를 처리한다.
    + 스칼라 라이브러리에서 Option 을 사용하는 예
        + Map 에서 주어진 키를 찾는 함수는 Option 을 돌려준다.
        + 목록에서 headOption 과 lastOption 을 사용한다. 
    + Option 은 왜 편리한가?
        + 고차 함수를 이용해 오류 처리의 공통 패턴을 추출할 수 있다.
        + 예외 처리에 수반되는 판에 박힌 코드를 작성하지 않아도 된다. 

In [70]:
trait Option[+A] {
    def map[B](f: A=>B) : Option[B] // Option 이 None 이 아니면, f 를 적용한다.
    def flatMap[B](f: A=>Option[B]) : Option[B] // Option 이 None 이 아니면 f 를 적용한다.
    def getOrElse[B >: A](default: => B) : B // Option 이 None 이면 default 를 None 이 아니면 
    def orElse[B>:A](ob: => Option[B]): Option[B]
    def filter(f:A=>Boolean) : Option[A] // 값이 f를 만족하지 않으면 Some 을 None 으로 변환한다.
}

defined trait Option

+ trait 로 정의하기 : 위에서 Option 에 대한 기본적인 함수들을 trait 로 정의하였다.
+ trait 로 정의하면 
    + obj.fn(arg) 나 obj fn arg1 같은 방식으로 호출할 수 있다. 
    + 동반객체로 정의하면 fn(obj,arg1) 과 같은 형식으로 호출해야 한다. 
+ Option 함수 정의에서 특이한 부분 살펴보기
    + default :=> 
        + 해당 인수의 형식이 B 이지만 그 인수가 함수에서 실제로 쓰일 때까지는 평가되지 않는다. 
        + 이를 비엄격성(non-strictness)라고 하는데, 5장에서 설명한다.
    + B >: A
        + B가 A 타입이거나 A 의 supertype 임을 의미한다.
        + Option[+A] 를 A 의 공변형식으로 선언해도 안전하다고 추론하게 하려면 반드시 이렇게 지정해야 한다.
+ 함수들의 정의
    + map \[ B \](f: A=>B) : Option[B]
        + 자신이 Some 인 경우에 변환을 수행한(함수 f를 적용) 후에, 변환된 값을 Some 으로 감싸서 돌려준다.
        + None 인 경우에는 None 을 반환한다.
    + flatMap \[ B \](f: A=>Option[B]) : Option[B]
        + Some 인 경우에 함수를 적용한 Option 값을 반환한다.
        + None 인 경우에는 None 을 돌려준다.
    + getOrElse \[ B >: A \](default: => B) : B
        + 자신이 None 이면 패러미터 값을 반환한다.
        + 자신이 None 이 아니면, 옵션 값을 반환한다.
    + orElse \[ B >: A \](ob: => Option[B]): Option[B]
        + getOrElse 와 같지만, Option 으로 돌려준다는 것만 다르다. 
    + filter(f:A => Boolean) : Option[A]
        + 필터로 전달된 함수가 성공할 때는 자기자신을 반환한다.
        + 그렇지 않으면, None 을 반환한다.

연습문제 4.1
+ Option 의 함수들을 모두 구현하라. 각 함수를 구현할 때 그 함수가 어떤 일을 하고 어떤 상황에서 쓰일 것인지 생각해 보라. 이 함수들 각각의 용도를 잠시 후에 보게 될 것이다.

In [71]:
//hide std library `Option` and `Either`, since we are writing our own in this chapter
import scala.{Option => _, Either => _, _}

sealed trait Option[+A] {
    def map[B](f: A=>B) : Option[B] = {
        this match {
            case None => None
            case Some(a) => println(a); Some(f(a))
        }
    }
    def getOrElse[B >: A](default: => B) : B = {
        this match {
            case None => default
            case Some(a) => a
        }
    }
    def flatMap[B](f: A=>Option[B]) : Option[B] = {
        this match {
            case None => None
            case Some(a) => f(a)
        }
    }
    def orElse[B>:A](ob: => Option[B]): Option[B] = {
        this map(Some(_)) getOrElse ob
    }
    
    def filter(f:A=>Boolean) : Option[A] = {
        this match {
            case Some(a) if f(a) => Some(a)
            case _ => None
        }
    }
}

case class Some[+A](get:A) extends Option[A]
case object None extends Option[Nothing]

import scala.{Option => _, Either => _, _}
defined trait Option
defined class Some
defined object None

In [10]:
case class Employee(name:String, department:String)

def lookupByName(name:String) : Option[Employee] = name match {
    case "Joe" => Some(new Employee("Joe","Development"))
    case _ => None
}

val joeDepartment: Option[String] = lookupByName("Joe").map(_.department)
val jaehoonDepartment: Option[String] = lookupByName("Jaehoon").map(_.department)

defined class Employee
defined function lookupByName
joeDepartment: scala.Option[String] = Some(Development)
jaehoonDepartment: scala.Option[String] = None

+ joeDepartment 
    + "Joe" 라는 이름을 가진 직원이 있는지 찾아서 부서를 돌려준다.
+ jaehoonDepartment
    + lookupByName 이 None 을 돌려주기 때문에 map 함수가 실행되지 않는다.

lookupByName("Joe").map(_.department)
lookupByName("Joe").flatMap(_.manager)
lookupByName("Joe").map(_.department).getOrElse("Default Dept.")

연습문제 4.2 
variance 함수를 flatMap 을 이용해서 구현하라. 순차열이 평균이 m 이라 할 떄, 순차열의 각 요소 x를 math.pow(x-m,2) 을 평균한 값이 분산이다.

def variance(xs: Seq \[ Double \]): Option\[ Double \]

In [73]:
val dept: String = 
    lookupByName("Joe").
    map(_.department).
    filter(_ != "Accounting").
    getOrElse("Default Dept")

: 

+ 오류를 보통의 값으로 돌려주었기 때문에 코드를 읽기가 훨씬 편하다.
    + 예외를 처리하기 위한 로직이 필요없기 때문에, 코드 짜기가 편해진다.
    + 고차함수를 사용해서 오류 처리 논리를 통합하면서도 비지니스 로직과는 격리할 수 있다.
    + 계산할 때마다 None 을 점검할 필요가 없다.
    + 타입 안정성을 얻을 수 있다. Option[A] 와 A 는 다른 타입이기 때문에 None 일 수 있는 상황을 처리하지 않으면 컴파일 오류가 발생한다.

### 4.3.2 예외 지향적 API 의 Option 승급, 감싸기

+ Option 을 사용하기 시작하면, 코드 기반 전체에 Option 이 번지게 되지 않을까?
    + 즉, Option 을 받거나 돌려주는 메소드를 호출하는 모든 코드를 Some 이나 None  을 처리하도록 수정해야 하지 않을까?
    + 그렇게 하지 않아도 된다. 
    + 보통 함수를 Option 에 대해서도 작용하는 함수로 승급(lift)시킬 수 있기 때문이다.
        + Option[A] 의 map 함수를 부르고
        + A => B 타입의 함수를 전달하면
        + Option[B] 를 돌려주는 함수로 바꿀 수 있다.

In [74]:
def lift[A,B](f: A=> B): Option[A] => Option[B] = _ map f

val abs2: Option[Double] => Option[Double] = lift(math.abs)

: 

+ lift 는 None 을 None 으로 사상하고 Some 의 내용을 f 에 적용하는 함수를 돌려준다. 
    + f 는 Option 타입을 인지할 필요가 없다.
+ Option 문맥으로 승급시키기
    + 위의 math.abs 처럼 선택적 값에 따라 작동하는 함수를 모두 작성할 필요가 없다.
    + 함수를 Option 문맥으로 승급시키기만 하면 된다.

In [21]:
def insuranceRateQuote(age:Int,numberOfSpeedingTickets:Int) : Double = 
    age * numberOfSpeedingTickets

def parseInsuranceRateQuote(age:String,numberOfSpeedingTickets: String) : Option[Double] = {
    val optAge: Option[Int] = Try(age.toInt)
    val optTickets : Option[Int] = Try(numberOfSpeedingTickets.toInt)
    insuranceRateQuote(optAge,optTickets)
}

def Try[A](a: => A) : Option[A] = {
    try Some(a)
    catch { case e: Exception => None }
}

: 

+ Try 는 예외기반 API 를 Option 지향적 API 로 변환하는 범용 함수다.
    + 이 함수는 게으른(lazy) 인수를 사용한다. 

연습문제 4.3
+ 두 Option 값을 이항 함수(binary function)을 이용해서 결합하는 일반적 함수 map2 를 작성하라.
+ 두 Option 값 중 하나라도 None 이면 map2 의 결과 역시 None 이어야 한다. 
+ Signature 는 다음과 같다.

def map2 \[ A,B,C \](a:Option\[ A \],b:Option \[ B \])(f:(A,B) => C) : Option \[ C \]

In [11]:
def map2[A,B,C](a:Option[A],b:Option[B])(f:(A,B) => C) : Option[C] =
    a flatMap ( aa => b map ( bb => f(aa,bb)))

defined function map2

+ map2 함수는 인수가 두 개인 함수를 아무 수정없이 Option 에 대응하게 만들 수 있다.

In [12]:
def insuranceRateQuote(age:Int,numberOfSpeedingTickets:Int) : Double = 
    age * numberOfSpeedingTickets

def parseInsuranceRateQuote(age:String,numberOfSpeedingTickets: String) : Option[Double] = {
    val optAge: Option[Int] = Try(age.toInt)
    val optTickets : Option[Int] = Try(numberOfSpeedingTickets.toInt)
    map2(optAge,optTickets)(insuranceRateQuote)
}

def Try[A](a: => A) : Option[A] = {
    try Some(a)
    catch { case e: Exception => None }
}

defined function insuranceRateQuote
defined function parseInsuranceRateQuote
defined function Try

연습문제 4.4
+ Option 들의 목록을 받고, 그 목록에 있는 모든 Some 값으로 구성된 목록을 담은 Option 을 돌려주는 함수 sequence 를 작성하라. 
+ 원래의 목록에 None 이 하나라도 있으면 함수의 결과도 None 이어야 한다.
+ 원래의 목록에 None 이 없으면, 원래 목록에 있는 모든 값의 목록을 담은 Some 을 돌려주어야 한다.
+ Signature 는 다음과 같다.

def sequence \[ A \](a: List \[ Option \[ A \] \]) : Option \[ List \[ A \] \]

In [20]:
def sequence [ A ](a: List [ Option [ A ] ]) : Option [ List [ A ] ] = {
    a match {
        case Nil => Some( Nil )
        case h :: t => h flatMap (hh => sequence(t) map (hh :: _))
    }
}

defined function sequence

+ 목록의 원소 중 하나라도 None 을 돌려주면, 전체 결과가 None 이 되게 해야 할 때도 있다.
+ 예를 들어, String 목록을 정수로 파싱하는데 하나라도 실패하면 전체가 실패하게 만들어야 한다면 어떻게 할까?

In [22]:
def parseInt(a:List[String]) : Option[List[Int]] =
    sequence( a map ( i => Try(i.toInt)) )

: 

+ 이 접근 방법은 목록을 두 번 훑어야 하기 때문에 비효율적이다. 
    + 한 번은, 각 String 을 Option \[ Int \] 로 변환하기 위해서
    + 또 한 번은, Option \[ Int \] 값들을 하나의 Option\[ List \[ Int \] \] 로 결합하기 위해서

연습문제 4.5
+ def traverse \[ A, B \](a: List \[ A \])(f: A => Option \[ B \]) : Option \[ List \[ B \] \]

+ for-함축

In [13]:
def map2[A,B,C](a:Option[A], b:Option[B])(f: (A,B) => C) : Option[C] =
    for {
        aa <- a
        bb <- b
    } yield f(aa,bb)

defined function map2

In [14]:
def map2[A,B,C](a:Option[A], b:Option[B])(f: (A,B) => C) : Option[C] =
    a flatMap ( 
        aa => b map (
            bb => f(aa,bb)
        )
    )

defined function map2

## 4.4 Either 자료 형식

+ Option 의 문제는?
    + 예외적인 조건이 발생했을 때 무엇이 잘못되었는지에 대한 정보를 제공하지 못한다.
    + 실패하면, 유효한 값이 없음을 나타내는 None 을 돌려줄 뿐이다. 
+ 예외가 발생했을 때 더 상세한 정보를 전달하고 싶다면 어떻게 할까?
    + 실패에 관해 알고 싶은 정보가 있다면, 실패 정보를 부호화하는 자료 형식을 만들 수 있다.
    + Either 타입을 사용하면 실패의 원인을 추적할 수 있다.

In [15]:
sealed trait Either[+E,+A]
case class Left[+E](value:E) extends Either[E,Nothing]
case class Right[+A](value:A) extends Either[Nothing,A]

defined trait Either
defined class Left
defined class Right

+ Either
    + Option 과 달리 두 경우 모두 다 값ㅇㄹ 가진다.
    + 두 값중 하나일 수 있는 타입을 나타낸다.
    + 이 타입은 두 타입의 분리 합집합(disjoint union)이라 할 수 있다 
    + 성공을 나타낼 때는 Right 를 사용하고
    + 실패를 나타낼 때는 Left 를 사용한다. 
    + 왼쪽 매개변수인 E 는 에러를 나타낸다. 

In [16]:
def mean(xs: IndexedSeq[Double]): Either[String,Double] = {
    if ( xs.isEmpty )
        Left("mean of empty list!")
    else Right(xs.sum / xs.length)
}

mean(IndexedSeq())

defined function mean
res13_1: cmd13.INSTANCE.$ref$cmd12.Either[String,Double] = Left(mean of empty list!)

+ 오류가 발생한 위치를 알고 싶다면 예외를 저장하면 된다.

In [17]:
def safeDiv(x:Int, y:Int): Either[Exception,Int] =
    try Right(x/y)
    catch { case e: Exception => Left(e)}

defined function safeDiv

In [18]:
def Try[A](a: => A) : Either[Exception,A] =
    try Right(a)
    catch { case e: Exception => Left(e) }

defined function Try

In [19]:
trait Either[+E, +A] {
    def map[B](f: A => B) : Either[E,B]
    def flatMap[EE >:E,B](f: A => Either[EE,B]): Either[EE,B]
    def orElse[EE >:E, B>:A](b: => Either[EE,B]): Either[EE,B]
    def map2[EE >:E, B, C](b:Either[EE,B])(f: (A,B) => C): Either[EE,C]
}

defined trait Either